In [4]:
### OLD ###
from nasem_dairy.ration_balancer.ration_balancer_functions import check_coeffs_in_coeff_dict
from nasem_dairy.ration_balancer.coeff_dict import coeff_dict
import math
from icecream import ic

def dry_cow_equations(DMIn_eqn, An_BW, An_PrePartWk, An_GestDay, An_GestLength, Dt_NDF, coeff_dict):

    req_coeffs = ['Ka_LateGest_DMIn', 'Kc_LateGest_DMIn']
    check_coeffs_in_coeff_dict(coeff_dict, req_coeffs)

    Dt_NDF_drylim = Dt_NDF
    #constrain Dt_NDF to the range of 30 to 55% of DM
    if Dt_NDF < 30:
        Dt_NDF_drylim = 30
    elif Dt_NDF > 55:
        Dt_NDF_drylim = 55

    #Late Gestation eqn. from Hayirli et al., 2003) for dry cows and heifers
    if An_PrePartWk < -3:
        An_PrePartWklim = -3
    elif An_PrePartWk > 0:
        An_PrePartWklim = 0
    else:
        An_PrePartWklim = An_PrePartWk
    An_PrePartWkDurat = An_PrePartWklim * 2



    Kb_LateGest_DMIn = -(0.365 - 0.0028 * Dt_NDF_drylim)
    Dt_DMIn_BW_LateGest_i = coeff_dict['Ka_LateGest_DMIn'] + Kb_LateGest_DMIn * An_PrePartWklim + coeff_dict['Kc_LateGest_DMIn'] * An_PrePartWklim**2
    Dt_DMIn_BW_LateGest_p = (coeff_dict['Ka_LateGest_DMIn'] * An_PrePartWkDurat + Kb_LateGest_DMIn / 2 * An_PrePartWkDurat**2 + coeff_dict['Kc_LateGest_DMIn'] / 3 * An_PrePartWkDurat**3) / An_PrePartWkDurat
    
    Dt_DMIn_DryCow1_FarOff = An_BW * Dt_DMIn_BW_LateGest_i / 100
    Dt_DMIn_DryCow1_Close = An_BW * Dt_DMIn_BW_LateGest_p / 100

    if An_PrePartWk > An_PrePartWkDurat:
        Dt_DMIn_DryCow1 = min(Dt_DMIn_DryCow1_FarOff, Dt_DMIn_DryCow1_Close)
    else:
        Dt_DMIn_DryCow1 = Dt_DMIn_DryCow1_FarOff

    ## from Hayirli et al., 2003 JDS
    if (An_GestDay - An_GestLength) < -21:
        Dt_DMIn_DryCow_AdjGest = 0
    else:
        Dt_DMIn_DryCow_AdjGest = An_BW * (-0.756 * math.exp(0.154 * (An_GestDay - An_GestLength))) / 100
    
    Dt_DMIn_DryCow2 = An_BW * 1.979 / 100 + Dt_DMIn_DryCow_AdjGest

    if DMIn_eqn == 10:
        return_DMI = Dt_DMIn_DryCow1
    elif DMIn_eqn == 11:
        return_DMI = Dt_DMIn_DryCow2
    # print(Dt_DMIn_BW_LateGest_p)
    print(Dt_DMIn_BW_LateGest_i)

    return return_DMI

dry_cow_equations(11, 600, -2, 180, 280, 14, coeff_dict)

1.892


11.874

In [2]:
### NEW ### 
from nasem_dairy.ration_balancer.ration_balancer_functions import check_coeffs_in_coeff_dict

# TODO add test, add execute
# Need when DMIn_eqn == 2,3,4,5,6,7,10
def calculate_Dt_DMIn_BW_LateGest_i(An_PrePartWklim, Kb_LateGest_DMIn, coeff_dict):
    req_coeffs = ['Ka_LateGest_DMIn', 'Kc_LateGest_DMIn']     
    check_coeffs_in_coeff_dict(coeff_dict, req_coeffs)
    #Late gestation individual animal prediction, % of BW.  Use to assess for a specific day for a given animal
    Dt_DMIn_BW_LateGest_i = coeff_dict['Ka_LateGest_DMIn'] + Kb_LateGest_DMIn * An_PrePartWklim + coeff_dict['Kc_LateGest_DMIn'] * An_PrePartWklim**2
    return Dt_DMIn_BW_LateGest_i


# TODO add test, add execute
# Need when DMIn_eqn == 10,12,13,14,15,16,17
def calculate_Dt_DMIn_BW_LateGest_p(An_PrePartWkDurat, Kb_LateGest_DMIn, coeff_dict):
    req_coeffs = ['Ka_LateGest_DMIn', 'Kc_LateGest_DMIn']     
    check_coeffs_in_coeff_dict(coeff_dict, req_coeffs)
    #Late gestation Group/Pen mean DMI/BW for an interval of 0 to PrePart_WkDurat.  Assumes pen steady state and PrePart_wk = pen mean
    Dt_DMIn_BW_LateGest_p = (coeff_dict['Ka_LateGest_DMIn'] * An_PrePartWkDurat + Kb_LateGest_DMIn / 2 * An_PrePartWkDurat**2 + coeff_dict['Kc_LateGest_DMIn'] / 3 * An_PrePartWkDurat**3) / An_PrePartWkDurat
    return Dt_DMIn_BW_LateGest_p


# TODO update tests, update execute
# Need when DMIn_eqn == 2,3,4,5,6,7
def calculate_Dt_DMIn_Heif_LateGestInd(An_BW, Dt_DMIn_BW_LateGest_i):
    #Individual intake for the specified day prepart or the pen mean intake for the interval, 0 to PrePart_WkDurat
    Dt_DMIn_Heif_LateGestInd = 0.88 * An_BW * Dt_DMIn_BW_LateGest_i / 100 #Individual animal
    return Dt_DMIn_Heif_LateGestInd
 

# TODO update tests, update execute
# Need when DMIn_eqn == 12,13,14,15,16,17
def calculate_Dt_DMIn_Heif_LateGestPen(An_BW, Dt_DMIn_BW_LateGest_p):
    Dt_DMIn_Heif_LateGestPen = 0.88 * An_BW * Dt_DMIn_BW_LateGest_p / 100 #Pen mean
    return Dt_DMIn_Heif_LateGestPen


# TODO add test, add execute
# DMIn_eqn == 10
def calculate_Dt_DMIn_DryCow1_FarOff(An_BW, Dt_DMIn_BW_LateGest_i):
    Dt_DMIn_DryCow1_FarOff = An_BW * Dt_DMIn_BW_LateGest_i / 100
    return Dt_DMIn_DryCow1_FarOff


# TODO add test, add execute
# DMIn_eqn == 10
def calculate_Dt_DMIn_DryCow1_Close(An_BW, Dt_DMIn_BW_LateGest_p):
    Dt_DMIn_DryCow1_Close = An_BW * Dt_DMIn_BW_LateGest_p / 100
    return Dt_DMIn_DryCow1_Close


# TODO add test, add execute
# DMIn_eqn == 11
def calculate_Dt_DMIn_DryCow2(An_BW, An_GestDay, An_GestLength):
    # from Hayirli et al., 2003 JDS
    if (An_GestDay - An_GestLength) < -21:
        Dt_DMIn_DryCow_AdjGest = 0
    else:
        Dt_DMIn_DryCow_AdjGest = An_BW * (-0.756 * math.exp(0.154 * (An_GestDay - An_GestLength))) / 100
    
    Dt_DMIn_DryCow2 = An_BW * 1.979 / 100 + Dt_DMIn_DryCow_AdjGest
    return Dt_DMIn_DryCow2


In [9]:
# calculate_Dt_DMIn_BW_LateGest_i(-3,-0.5, coeff_dict)
#2.655

# calculate_Dt_DMIn_BW_LateGest_p(-2, -0.3, coeff_dict)
# 1.7233333333333334

# calculate_Dt_DMIn_DryCow1_FarOff(820, 1.9)
#15.58

calculate_Dt_DMIn_DryCow1_Close(755, 1.7)
# 12.835

# calculate_Dt_DMIn_DryCow2(600, 180, 280)


12.835